In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\96653\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\96653\AppData\Roaming\nltk_data...


True

In [3]:
df = pd.read_csv('../data/pif_companies_filtered.csv')

In [3]:
df

,Title,Description
0,Alat,Alat is a company focused on transforming glob...
1,Dan Company,Dan Company is a Public Investment Fund (PIF) ...
2,ARDARA,ARDARA is a real estate developer owned by the...
3,TASARU Mobility Investments,TASARU Mobility Investments is the investment ...
4,Albalad Development CO.,"Al Balad Development Company (BDC), wholly own..."
...,...,...
92,Sela Sport Company,"Established in 1997, Sela is Saudi Arabia's in..."
93,Electric Vehicle Infrastructure Company (EVIQ),The Electric Vehicle Infrastructure Company (E...
94,Kuwaiti Food Co. (Americana),"Americana Restaurants, established in 1964 in ..."
95,Riyad Bank,Riyad Bank is one of the largest financial ins...


In [4]:
nulls_values = df.isna().any(axis=1)
nulls_values
nulls_values.sum()

0

In [5]:
## Change The Title and Description  to lower()
df['Title'] = df['Title'].str.lower()
df['Description'] = df['Description'].str.lower()

In [6]:
# Remove any Special Characters


def clean_text(text):
    if isinstance(text, str):
        return re.sub(r'[^\w\s]', '', text).strip()
    return text

df['Title'] = df['Title'].apply(clean_text)
df['Description'] = df['Description'].apply(clean_text)


In [7]:
df

,Title,Description
0,alat,alat is a company focused on transforming glob...
1,dan company,dan company is a public investment fund pif co...
2,ardara,ardara is a real estate developer owned by the...
3,tasaru mobility investments,tasaru mobility investments is the investment ...
4,albalad development co,al balad development company bdc wholly owned ...
...,...,...
92,sela sport company,established in 1997 sela is saudi arabias inau...
93,electric vehicle infrastructure company eviq,the electric vehicle infrastructure company ev...
94,kuwaiti food co americana,americana restaurants established in 1964 in k...
95,riyad bank,riyad bank is one of the largest financial ins...


In [8]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

##

def process_text(text):
    if isinstance(text, str):

        words = text.split()  ## Split The Data

        stop_words = set(stopwords.words('english')) - {'company', 'group', 'investment', 'bank', 'development', 'saudi', 'international'}  # Remove common words and some words need it to stay

        words = [word for word in words if word not in stop_words]  ## Loop for words

        lemmatizer = WordNetLemmatizer()  # Initialize the lemmatizer

        words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize the words

        return ' '.join(words)

    return text

# Apply the function to both Title and Description columns
df['Title'] = df['Title'].apply(process_text)
df['Description'] = df['Description'].apply(process_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# data['Description'].fillna('', inplace=True)
# data['Description'] = data['Description'].str.lower()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Description'])

def search_companies(query, top_n=5):
  query_vec = vectorizer.transform([query.lower()]) # query to lower case
  similarity_scores = np.dot(query_vec, tfidf_matrix.T).toarray()[0]
  top_indices = np.argsort(similarity_scores)[::-1][:top_n]
  results = df.iloc[top_indices]
  results['similarity_score'] = similarity_scores[top_indices]
  return results


search_results = search_companies("when was Riyadh air lunched?", top_n=3)
search_results

<ipython-input-13-cd0297aae653>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity_score'] = similarity_scores[top_indices]


,Title,Description,similarity_score
12,riyadh air,riyadh air worldclass airline owned public inv...,0.459863
38,king abdullah financial district kafd,situated riyadh kafd iconic business lifestyle...,0.083792
15,king salman international airport,king salman international airport developed ca...,0.072040
